In [1]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from guidedlda import GuidedLDA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

print('ok')

ModuleNotFoundError: No module named 'guidedlda'

In [ ]:


# 示例数据，你需要替换为你的实际数据
corpus = [
    {"text": "This is a positive review about a product.", "label": "positive"},
    {"text": "Negative feedback on the service received.", "label": "negative"},
    # Add more samples as needed
]

# 将数据转化为DataFrame格式
import pandas as pd
df = pd.DataFrame(corpus)

# 数据预处理
df = shuffle(df, random_state=42).reset_index(drop=True)  # 随机打乱数据
X = df['text']
y = df['label']

# 将标签编码为数字
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 将文本转化为词袋表示
vectorizer = CountVectorizer()
X_vec = vectorizer.fit_transform(X)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_vec, y_encoded, test_size=0.2, random_state=42)

# GuidedLDA模型
model = GuidedLDA(n_topics=2, n_iter=100, random_state=42)
# 在这里，你需要提供一些关键词作为每个主题的引导词
seed_topics = {'positive': ['positive', 'good', 'excellent', 'satisfied'],
               'negative': ['negative', 'poor', 'bad', 'unsatisfied']}
model.fit(X_train.toarray(), seed_topics=seed_topics, seed_confidence=0.15)

# 输出主题-词分布
n_top_words = 10
topic_word = model.topic_word_
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vectorizer.get_feature_names())[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ', '.join(topic_words)))

# 在测试集上进行预测
y_pred = model.transform(X_test.toarray()).argmax(axis=1)

# 计算准确率
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
